# A quick tour of StatsModels.jl
### Dave Kleinschmidt
### 18 February 2020

In [61]:
using DataFrames, Pipe
using MixedModels, GLM
using LinearAlgebra, Statistics

using StatsModels
using StatsModels: pretty_mat

# Contrast coding

## What and why?

To fit any kind of statistical model you need some kind of *numerical
representation* of your data. Data often comes in a *table*, a named
collection of variables of different types of data. Some of that data is
"continuous", or basically numeric. But often our data is not numeric
(or continuous), but "categorical", having a finite number of distinct
levels.

For instance, let's look at the KB07 dataset:

In [ ]:
kb07 = MixedModels.dataset(:kb07)
first(kb07, 5)

Here `:spkr`, `:prec`, and `:load` are categortical variables, each of
which takes on two different values. If we fit a regression using this
dataset, we end up with predictors that refer to specific levels:

In [ ]:
f = @formula(rt_trunc ~ 1 + spkr + prec + spkr&prec + (1 | subj))
mod = fit(MixedModel, f, kb07)

Let's look at a few rows of the fixed effects design matrix that's
generated for this model:

In [ ]:
pretty_mat(mod.X[1:5, :])

A few things to note: all the values are 0 or 1, and there's one column
of all 1s at the start (that's the `(Intercept)` term). Columns 2 and 3
correspond to `spkr` and `prec`: there's a 0 where `spkr == "new"` and a
1 for `"old"`. Note that the coefficient name for this column is `spkr:
old`, which indicates that this predictor indicates the presence of
"old", relative to the (implicit) baseline of "new". Similarly for
`prec: maintain`.

The last column is the interaction term `spkr&prec`, and it's the
elementwise product of the columns for `spkr: new` and `pred: maintain`.

## How to take control

You can set your own contrasts via the `contrasts=` keyword argument in
`fit`, with the variable you want to code as the key and contrasts as
the value:

In [ ]:
using StatsModels

contrasts = Dict(
    :spkr => EffectsCoding(base = "old"),
    :prec => DummyCoding(levels = ["maintain", "break"])
)

mod2 = fit(MixedModel, f, kb07, contrasts=contrasts)

This example illustrates two ways to control the ordering of levels used
to compute the contrasts:

1.  you can use `base=` to determine the baseline level
2.  you can use `levels=` to indicate all the levels that are used in
    the contrasts, the first of which is automatically set as the
    baseline.

### Reversed Helmert coding

Let's say you want to use reverse Helmert coding.  It's easy using `reverse` to
flip the order of the levels.  Here's the original:

In [ ]:
spkr_levels = ["old","new"]
fit(MixedModel,
    f,
    kb07,
    contrasts = Dict(:spkr => HelmertCoding(levels=spkr_levels)))

And reversed:

In [ ]:
fit(MixedModel,
    f,
    kb07,
    contrasts = Dict(:spkr => HelmertCoding(levels=reverse(spkr_levels))))

## Built in contrast coding schemes

StatsModels.jl provides a few commonly used contrast coding schemes,
some less-commonly used schemes, and structs that allow you to manually
specify your own, custom schemes.

All are subtypes of the `AbstractContrasts` type:

In [ ]:
using InteractiveUtils
subtypes(AbstractContrasts)

And all have fairly extensive documentation via the normal help system.
For instance:

In [ ]:
# use ?SeqDiffCoding in the REPL

### Standard contrasts

The most commonly used contrasts are `DummyCoding` and `EffectsCoding`
(which are similar to `contr.treatment` and `contr.sum` in R,
respectively).

### "Exotic" contrasts

We also provide `HelmertCoding` and `SeqDiffCoding` (corresponding to
base R's `contr.helmert` and MASS's `contr.sdiff`).

## Manual contrasts

There are two ways to manually specify contrasts. First, you can specify
them **directly** via `ContrastsCoding`. If you do, it's good practice
to specify the levels corresponding to the rows of the matrix, although
they can be omitted in which case they'll be inferred from the data.

For instance, here's a weird set of contrasts for `:spkr`:

In [ ]:
cs = Matrix([-1/3 2/3]')
contr_manual = Dict(:spkr => StatsModels.ContrastsCoding(cs, levels=["old", "new"]))
mod3 = fit(MixedModel, f, kb07, contrasts=contr_manual)

(Note that the estimates and even the signs of the fixed effect βs
change when we change the contrasts, but the overall log-likelihood
doesn't).

We can see that the values from the contrasts matrix we specified are
plugged directly in to the fixed effects matrix, and are also used in
computing the predictor for the interaction:

In [ ]:
mod3.X[1:5, :]

### Example: manual Helmert contrasts

Let's say you want Helmert contrasts but you always forget what it's called.
Here's how you can manually specify them using `StatsModels.ContrastsCoding`.

Because this isn't very interesting with only two levels, let's combine `:spkr`
and `:prec` into a single, 4-level variable:

In [ ]:
kb07ex = transform(kb07, AsTable([:spkr, :prec]) => (x -> x.spkr .* "-" .* x.prec) => :spkr_prec);
levels = ["new-break", "new-maintain", "old-break", "old-maintain"]
f2 = @formula(rt_trunc ~ 1 + spkr_prec + (1 | subj))

In [ ]:
using StatsModels: ContrastsCoding
man_helm = [-1 -1 -1
             1 -1 -1
             0  2 -1
             0  0  3]
contr_helm_man = ContrastsCoding(man_helm[:,1:3], levels=levels)
fit(MixedModel, f2, kb07ex, contrasts = Dict(:spkr_prec => contr_helm_man))

We can see that this is equivalent to `HelmertCoding`:

In [ ]:
fit(MixedModel, f2, kb07ex, contrasts = Dict(:spkr_prec => HelmertCoding(levels=levels)))

## Contrasts from hypotheses

A better way to specify manual contrasts is via `HypothesisCoding`, where each
row of the matrix corresponds to the weights given to the cell means of the
levels corresponding to each column (see [Schad et
al. 2020](https://doi.org/10.1016/j.jml.2019.104038) for more information). As
before with manual contrasts, this is less interesting with only two levels, so
we'll again look at a scenario where we combine `:spkr` and `:prec` into a
single, 4-level predictor, and want to test some strange hypotheses.

Here's the model fit with the default (dummy/treatment-coded contrasts):

In [ ]:
mod4 = fit(MixedModel, f2, kb07ex)

Let's see how you could generate custom contrasts for a number of different a
priori hypotheses.

### Example: Sequential differences coding

One hypothesis you might want to test is that the first condition is different
from the second, the second from the third, the third from the fourth, etc.
First we have to turn these hypotheses into a numeric form.  The null hypothesis
that condition 1 is not different from condition 2 can be expressed by saying
the difference between the two mean responses in these conditions is zero.
Mathematically, we can write that as ``\mu_2 - \mu_1 = 0``, or equivalently:

```math
-1 \cdot \mu_1 + 1 \cdot \mu_2 + 0 \cdot \mu_3 + ... + 0 \cdot \mu_n = 0
```

The weights for each of the means are the entries in our hypothesis vector for
this hypothesis.  So the first hypothesis vector is `[-1, 1, 0, 0]`.  Likewise,
the second is `[0, -1, 1, 0]` (``\mu_3 - \mu_2 = 0``), and the third is `[0, 0,
-1, 1]` (``\mu_4 - \mu_3 = 0``).  Putting these together we get:

In [ ]:
seq_diff_hyps = [-1  1  0  0
                  0 -1  1  0
                  0  0 -1  1]

These hypotheses correspond to the following *contrasts* (using the
[`StatsModels.pretty_mat`](https://github.com/JuliaStats/StatsModels.jl/blob/master/src/contrasts.jl#L716-L723)
function to make pretty fractions; this is based on
[`rationalize`](https://docs.julialang.org/en/v1/base/math/#Base.rationalize)
function, which is like the `fractions` function in R):

In [ ]:
seq_diff_contrs = HypothesisCoding(seq_diff_hyps, levels = levels)
pretty_mat(seq_diff_contrs.contrasts)

You can see that the contrasts for these hypothese are rather different!  It's
not immediately obvious just looking at them how they're related (at least not
to me), which shows the power of hypothesis coding: you can work in a format
that *does* make intuitive sense (the weights assigned to each group's mean
response).

When we fit the model, we should see that the corresponding betas are the same
as the differences between the cell means:

In [ ]:
fit(MixedModel,
    f2,
    kb07ex,
    contrasts = Dict(:spkr_prec => seq_diff_contrs))

Calculating the cell mean differences manually:

In [ ]:
cell_means = @pipe kb07ex |>
    groupby(_, :spkr_prec) |>
    combine(_, :rt_trunc => mean => :mean_rt) |>
    innerjoin(DataFrame(spkr_prec=levels), _, on=:spkr_prec)  # make sure ordering is right
diff(cell_means.mean_rt)                                      # compare with betas above...

Note that the intercept corresponds to the *grand mean*, not to the first
level's mean!  That's because the hypothesis vectors are *zero-mean*, so they
don't affect the hypothesis for the intercept (as long as the design is
balanced).

### Example: custom, a priori hypotheses

Let's say we want to test whether the effect of `:prec` depends on
whether `:spkr` is old vs. new. We need one contrast to test the
hypothesis that `"maintain" != "break"` for "new", and another for
"old". That leaves one over, to test the overall difference between
"new" and "old".

In [ ]:
prec_old = (levels .== "old-break") .- (levels .== "old-maintain")

In [ ]:
prec_new = (levels .== "new-break") .- (levels .== "new-maintain")

In [ ]:
old_new = (abs.(prec_old) .- abs.(prec_new)) ./ 2

In [ ]:
contr_hyp = HypothesisCoding(hcat(old_new, prec_old, prec_new)',
                             labels=["old", "(old) break", "(new) break"])
contr_hyp.hypotheses

These hypotheses correspond to the following *contrasts*:

In [ ]:
pretty_mat(contr_hyp.contrasts)

Notice how the ±1 coding in the hypotheses (which translates into the difference
between the mean response in those cells) is transformed into ±½ coding in the
contrasts.

In [ ]:
mod5 = fit(MixedModel, f2, kb07ex, contrasts = Dict(:spkr_prec => contr_hyp))

Note that this is equivalent to the `/` "nesting" syntax using `EffectsCoding`,
after adjusting for the 2× factor from the +1/-1 coding:

In [ ]:
mod6 = fit(MixedModel,
           @formula(rt_trunc ~ 1 + spkr/prec + (1|subj)), 
           kb07,
           contrasts = Dict(:spkr => EffectsCoding(base="new"),
                            :prec => EffectsCoding(base="maintain")))

### Example: Helmert contrasts that actually make sense

Let's say you want something like Helmert contrasts, but where the βs are
interpretable as the difference between the $n$th level and the average of
levels $1\ldots n-1$.  Here are the hypotheses that correspond to that:

In [ ]:
helmert_hypotheses = [-1 -1/2 -1/3
                       1 -1/2 -1/3
                       0  1   -1/3
                       0  0    1]

And the resulting contrasts matrix:

In [ ]:
contr_helm_hyp = HypothesisCoding(helmert_hypotheses',
                                  levels=levels, labels=levels[2:end])
pretty_mat(contr_helm_hyp.contrasts)

Which is similar but not identical to the contrats matrix for HelmertCoding!  In
a way that I would not be able to derive off the top of my head.

Now we fit the model:

In [ ]:
fit(MixedModel, f2, kb07ex, contrasts = Dict(:spkr_prec => contr_helm_hyp))

...and we can see that the βs for levels are very close to the cumulative means
minus the mean for that level, computed manually

In [ ]:
lev_means = @pipe kb07ex |>
    groupby(_, :spkr_prec) |>
    combine(_, :rt_trunc => mean => :mean_rt) |>
    innerjoin(DataFrame(spkr_prec=levels), _, on=:spkr_prec) |>
    transform(_, :mean_rt => (x -> cumsum(x) ./ (1:length(x))) => :cum_mean) |>
    transform(_, [:mean_rt, :cum_mean] => ((x,y) -> x - lag(y)) => :diff_with_last_mean)

# The `@formula`

A formula in Julia is created with the `@formula` macro.  Between the macro and
fitting a model, the formula goes through a number of steps.

* The `@formula` macro itself does some transformations on the syntax, and
  creates `Term`s
* Then a `Schema` is extracted from the data, which says which `Term`s are
  `ContinuousTerm`s and which are `CategoricalTerm`s
* The `Schema` is then used to transform the original formula into a "concrete
  formula".
* The concrete formula (with all `Term`s replaced by continuous/categorical
  versions) generates model matrix columns when given some data.

The details are described in the
[documentation](https://juliastats.org/StatsModels.jl/stable/internals/#The-lifecycle-of-a-@formula-1),
and for the most part modeling packages handle these steps for you.  But in the
interest of allowing you to do your own weird things, here are a few examples.

## A formula is made of terms

In [ ]:
f = @formula(y ~ 1 + a + b + a&b)

You can inpsect the internal structure with (it's like `str` in R):

In [ ]:
dump(f)

We can build the same formula directly, using terms:

In [ ]:
t_a = term(:a)
t_b = term(:b)
t_1 = term(1)
t_y = term(:y)

dump(t_a)

In [ ]:
f_dir = FormulaTerm(t_y, (t_1, t_a, t_b, InteractionTerm((t_a, t_b))))

Or, using operator overloading:

In [ ]:
f_op = t_y ~ t_1 + t_a + t_b + t_a & t_b

These three are all equivalent:

In [ ]:
f == f_dir == f_op

## The schema gives concrete terms

If we have some fake data:

In [ ]:
df = DataFrame(y = rand(100), a = rand(100), b = repeat([:Q, :R, :S, :T], 25))
first(df, 5)

We can extract a `Schema`:

In [ ]:
sch = schema(df)

This maps un-typed `Term`s to concrete verisons.  Now we know that `:a` is a
continuous variable in this dataset:

In [ ]:
sch[term(:a)]

Categorical terms hold the contrasts matrix and levels:

In [ ]:
t_b_concrete = sch[term(:b)]

In [ ]:
cmat = t_b_concrete.contrasts
cmat.matrix

In [ ]:
cmat.levels

## `apply_schema` combines terms and schema to get concrete versions

The canonical case is to apply the schema to the whole formula:

In [ ]:
apply_schema(f, sch)

Note that the schema gets pushed through the interaction term, too.

We can also apply the schema to a single term:

In [ ]:
apply_schema(term(:a) & term(:b), sch)

Of course if the schema doesn't have enough information, we'll get an error:

In [ ]:
apply_schema(term(:argle_bargle), sch)

## Concrete terms generate arrays with `modelcols`

Any term can generate model columns with `modelcols`:

In [ ]:
t_ab_concrete = apply_schema(term(:a) & term(:b), sch)
modelcols(t_ab_concrete, first(df, 6))

Compare with:

In [ ]:
f_concrete = apply_schema(f, sch)
@show t_ab_concrete_formula = f_concrete.rhs.terms[end]
modelcols(t_ab_concrete_formula, first(df, 6))

Of course you can generate columns for the whole formula (it returns a tuple of
left-hand side, right-hand side columns):

In [ ]:
y, X = modelcols(f_concrete, first(df, 6))
X

### Predicting based on new data

Any table with the right columns can be passed to `modelcols` and the right
columns are generated, even if some levels are missing:

In [ ]:
df2 = DataFrame(a = rand(5), b = [:R, :R, :Q, :S, :R])
modelcols(f_concrete.rhs, df2)

### Use a named tuple for a single row

A single row of the model matrix can be generated from a `NamedTuple` of data:

In [ ]:
data_row = (a = 1.5, b = :T)
modelcols(f_concrete.rhs, data_row)

## Get coefficient names for any term with `coefnames`

In [ ]:
coefnames(f_concrete)

In [ ]:
coefnames(f_concrete.rhs.terms[end])

In [ ]:
coefnames(sch[term(:b)])

## Formula syntax

The formula syntax is very similar to R, with the exception that an interaction
is specified with `&`, and that some R syntax is not supported by default (`^`,
`/` outside of MixedModels.jl).

### Non-special calls 

Any function calls that are not special syntax (`+`, `&`, `*`, and `~`) are
treated as normal julia code, so you can write things like

In [ ]:
f2 = @formula(log(y) ~ 1 + (a + a^2) * b)

In [ ]:
f2_concrete = apply_schema(f2, sch)

In [ ]:
y2, X2 = modelcols(f2_concrete, first(df, 5))

In [ ]:
y2 == log.(df[1:5, :y])

In [ ]:
X2

In [ ]:
coefnames(f2_concrete.rhs)

### Advanced: making the ordinary special

You may have noticed that `zercocorr` and `|` were not included in the list of
special syntax above.  StatsModels.jl provides a method to add special syntax
for the `@formula` that's specific to certain models.  This works using the
standard Julia techniques of multiple dispatch, by providing methods that
intercept `apply_schema` for particular combinations of functions, schema, and
context (model type), like so:

```
function StatsModels.apply_schema(
    t::FunctionTerm{typeof(|)},
    schema::StatsModels.FullRank,
    Mod::Type{<:MixedModel},
)
    schema = StatsModels.FullRank(schema.schema)
    lhs, rhs = t.args_parsed
    if !StatsModels.hasintercept(lhs) && !StatsModels.omitsintercept(lhs)
        lhs = InterceptTerm{true}() + lhs
    end
    lhs, rhs = apply_schema.((lhs, rhs), Ref(schema), Mod)
    RandomEffectsTerm(MatrixTerm(lhs), rhs)
end
```

There's a simpler [example in the StatsModels
docs](https://juliastats.org/StatsModels.jl/stable/internals/#An-example-of-custom-syntax:-poly-1)
which adds a `poly(x, n)` syntax for polynomial regression.

### Example: specifying many different models

Let's see how each of the predictors in the KB07 dataset does on its own.

In [ ]:
template = @formula(rt_trunc ~ 1 + (1|subj) + (1|subj))
fits = map([:spkr, :prec, :load]) do p
    f = template.lhs ~ template.rhs + term(p)
    fit(MixedModel, f, kb07)
end

Which predictor provides the best fit to the data on its own?

In [ ]:
sort!(fits, by=objective)
foreach(fits) do fit
    println(round(fit.objective), ": ", fit.formula)
end

Looks like it's `prec`, followed by `load`, and then `spkr`.